In [1]:
import pandas as pd
from sportsreference.ncaab.teams import Teams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt
import urllib.request


from datetime import datetime
import requests
import os
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('NBA_2016_thru_2022_seasons.csv')

In [3]:
df

,Unnamed: 0,G,Date,Start (ET),Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,teamgame,season
0,0,1,"Tue, Oct 27, 2015",8:00p,Box Score,NaN,Detroit Pistons,L,NaN,94.0,106.0,0.0,1.0,L 1,ATL,2016
1,1,2,"Thu, Oct 29, 2015",8:00p,Box Score,@,New York Knicks,W,NaN,112.0,101.0,1.0,1.0,W 1,ATL,2016
2,2,3,"Fri, Oct 30, 2015",8:00p,Box Score,NaN,Charlotte Hornets,W,NaN,97.0,94.0,2.0,1.0,W 2,ATL,2016
3,3,4,"Sun, Nov 1, 2015",2:00p,Box Score,@,Charlotte Hornets,W,NaN,94.0,92.0,3.0,1.0,W 3,ATL,2016
4,4,5,"Tue, Nov 3, 2015",7:30p,Box Score,@,Miami Heat,W,NaN,98.0,92.0,4.0,1.0,W 4,ATL,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16587,77,78,"Sun, Apr 3, 2022",1:00p,NaN,@,Boston Celtics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WAS,2022
16588,78,79,"Tue, Apr 5, 2022",8:00p,NaN,@,Minnesota Timberwolves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WAS,2022
16589,79,80,"Wed, Apr 6, 2022",8:00p,NaN,@,Atlanta Hawks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WAS,2022
16590,80,81,"Fri, Apr 8, 2022",7:00p,NaN,NaN,New York Knicks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WAS,2022
